In [9]:
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from nltk.tokenize import sent_tokenize, word_tokenize

In [4]:
"""
Download dataset SubtaskA.jsonl from 
https://github.com/mbzuai-nlp/M4GT-Bench.
"""
DATA_PATH = "/home/javen/Documents/Datasets/M4GT-Bench/SubtaskA.jsonl"

# initialize dataset
df = pd.read_json(DATA_PATH, lines=True)
df = df[['text', 'label', 'model']]
dataset = Dataset.from_pandas(df)

# split dataset
a = dataset.train_test_split(test_size=0.20)
b = a['test'].train_test_split(test_size=0.5)
dataset = DatasetDict({
    'train': a['train'],
    'valid': b['train'],
    'test': b['test'],
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'model'],
        num_rows: 122247
    })
    valid: Dataset({
        features: ['text', 'label', 'model'],
        num_rows: 15281
    })
    test: Dataset({
        features: ['text', 'label', 'model'],
        num_rows: 15281
    })
})


In [14]:
"""
Tokenize dataset.
"""
def tokenize(X):
    X['text'] = [word_tokenize(sentence.lower()) for sentence in X['text']]
    return X

# tokenize data
tokenized_datasets = dataset.map(tokenize, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/122247 [00:00<?, ? examples/s]

Map:   0%|          | 0/15281 [00:00<?, ? examples/s]

Map:   0%|          | 0/15281 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'model'],
        num_rows: 122247
    })
    valid: Dataset({
        features: ['text', 'label', 'model'],
        num_rows: 15281
    })
    test: Dataset({
        features: ['text', 'label', 'model'],
        num_rows: 15281
    })
})
